In [2]:
import folium
import pandas as pd
from folium.plugins import HeatMap
import time
from IPython.display import display, clear_output

# Let's import our data
df = pd.read_csv('taxi_data.csv')

# Convert the tpep_pickup_datetime column to a datetime object
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Group the data by pickup_zone and tpep_pickup_datetime (hourly)
grouped_data = df.groupby([df['pickup_zone'], df['tpep_pickup_datetime'].dt.hour]).size().reset_index(name='count')

# Rename the columns for clarity
grouped_data.rename(columns={'tpep_pickup_datetime': 'hour'}, inplace=True)

# Let's merge with the zone data to get the longitude and latitude
grouped_data = pd.merge(grouped_data, df[['pickup_zone', 'pickup_lat', 'pickup_lon']].drop_duplicates(), on='pickup_zone', how='left')


# Create a map centered around New York City
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=20)

# Define a function to update the heatmap based on the selected hour
def update_heatmap(hour):
    nyc_map = folium.Map(location=[40.7528, -74.0060], zoom_start=12)
    heat_data = [[row['pickup_lat'], row['pickup_lon'], row['count']] for index, row in grouped_data[grouped_data['hour'] == hour].iterrows()]
    HeatMap(heat_data).add_to(nyc_map)
    folium.map.Marker(
        [40.8130, -74.1420],
        icon=folium.DivIcon(html=f'''
            <div style="background-color: white; border: 2px solid black; border-radius: 10px; padding: 10px; font-size: 24pt; width: 105px;">
                {hour}:00
            </div>
        ''')
    ).add_to(nyc_map)
    clear_output(wait=True)
    display(nyc_map)

# Loop through each hour and update the heatmap
for hour in range(24):
    update_heatmap(hour)
    time.sleep(1)  # Wait for 1 second before updating to the next hour


KeyboardInterrupt: 